# Assignment 3: Improving the Index

In this assignment, we will improve the search index and query functions from the previous assignment.

## Loading the Data and Defining Auxiliary Functions

This section is copied from the previous notebook.

In [1]:
import pickle, bz2, re
from collections import namedtuple, defaultdict, Counter
from IPython.display import display, HTML
from math import log10, sqrt

Summaries_file = 'data/allergy_Summaries.pkl.bz2'
Abstracts_file = 'data/allergy_Abstracts.pkl.bz2'

Summaries = pickle.load( bz2.BZ2File( Summaries_file, 'rb' ) )
Abstracts = pickle.load( bz2.BZ2File( Abstracts_file, 'rb' ) )

paper = namedtuple( 'paper', ['title', 'authors', 'year', 'doi'] )
for (id, paper_info) in Summaries.items():
    Summaries[id] = paper( *paper_info )

In [2]:
def tokenize(text):
    """
    Function that tokenizes a string in a rather naive way. Can be extended later.
    """
    return text.split(' ')

def preprocess(tokens):
    """
    Perform linguistic preprocessing on a list of tokens. Can be extended later.
    """
    result = []
    for token in tokens:
        result.append(token.lower())
    return result

In [3]:
def display_summary( id, show_abstract=False, show_id=True, extra_text='' ):
    """
    Function for printing a paper's summary through IPython's Rich Display System.
    Trims long author lists, and adds a link to the paper's DOI (when available).
    """
    s = Summaries[id]
    lines = []
    title = s.title
    if s.doi != '':
        title = '<a href=http://dx.doi.org/{:s}>{:s}</a>'.format(s.doi, title)
    title = '<strong>' + title + '</strong>'
    lines.append(title)
    authors = ', '.join( s.authors[:20] ) + ('' if len(s.authors) <= 20 else ', ...')
    lines.append(str(s.year) + '. ' + authors)
    if (show_abstract):
        lines.append('<small><strong>Abstract:</strong> <em>{:s}</em></small>'.format(Abstracts[id]))
    if (show_id):
        lines.append('[ID: {:d}]'.format(id))
    if (extra_text != ''):
         lines.append(extra_text)
    display( HTML('<br>'.join(lines)) )

In [4]:
inverted_index = defaultdict(set)

for (id, abstract) in Abstracts.items():
    for term in preprocess(tokenize(abstract)):
        inverted_index[term].add(id)

## Stemming

As we could see from the results of the last assignment, our simple index doesn't handle punctuation and the difference between singular and plural versions of the same word very well. We won't go much into the details of tokenization and linguistic analysis here, because we also want to focus on scoring and ranking below. Therefore, we are using an existing library for tokenizatoin and stemming, namely the NLTK package. The following line will install NLTK if necessary (or you have to follow [these instructions](http://www.nltk.org/install.html) if that doesn't work):

In [5]:
! pip install --user nltk

In [6]:
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import EnglishStemmer
import nltk
nltk.download('punkt')
stemmer = EnglishStemmer()

s = '''Good muffins cost $3.88\nin New York.  Please buy me two of them.\n\nThanks.'''

print('INPUT TEXT:\n ', s)

print('TOKENIZE: ', tokenize(s))
print('WORD TOKENIZE: ', word_tokenize(s))

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/andrewphilipharrison/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


INPUT TEXT:
  Good muffins cost $3.88
in New York.  Please buy me two of them.

Thanks.
TOKENIZE:  ['Good', 'muffins', 'cost', '$3.88\nin', 'New', 'York.', '', 'Please', 'buy', 'me', 'two', 'of', 'them.\n\nThanks.']
WORD TOKENIZE:  ['Good', 'muffins', 'cost', '$', '3.88', 'in', 'New', 'York', '.', 'Please', 'buy', 'me', 'two', 'of', 'them', '.', 'Thanks', '.']


In [43]:
print(stemmer.stem("signals"))

signal


## Ranking

Another important method to improve our search results is to rank them, which can be done by calculating a score for each document based on the matching terms from the query. One such scoring method is *tf-idf*, which comes with several variants, as explained in the lecture slides.

In order to quickly calculate the scores for a term/document combination, we'll need quick access to a couple of things:

- tf(t,d): How often does a term occur in a document
- df(t): In how many documents does a term occur
- num_documents: The number of documents in our index

In [8]:
tf_matrix = defaultdict(Counter)

for (doc_id, abstract) in Abstracts.items():
    tokens = preprocess(tokenize(abstract))
    tf_matrix[doc_id] = Counter(tokens)

def tf(t,d):
    return float(tf_matrix[d][t])

def df(t):
    return float(len(inverted_index[t]))

num_documents = float(len(Abstracts))

Let's test these functions with some examples:

In [9]:
print(tf('amsterdam', 16513592))
print(df('amsterdam'))
print(num_documents)

1.0
3.0
73679.0


With these helper functions, we can now easily calculate the _tf-idf_ weights of a term in a document by implementing the weighting formula from the slides, which you will do in the assignments below.

----------

# Tasks

**Your name:** Andrew Harrison

### Task 1

Implement in the code block below the `smarter_tokenize` function using NLTK's function for tokenization, and the `smarter_preprocess` function to perform stemming in addition to case normalization. 

In [31]:
# Smarter linguistic processing

# Your code here:

def smarter_tokenize(text):
    return word_tokenize(text)

def smarter_preprocess(tokens):
    result = []
    for token in tokens:
        result.append(stemmer.stem(token))
    return result

# To test it:
print(smarter_preprocess(smarter_tokenize("He buys many books, some about I.R., for less than $1.50!")))

['he', 'buy', 'mani', 'book', ',', 'some', 'about', 'i.r.', ',', 'for', 'less', 'than', '$', '1.50', '!']


Now we can make a smarter index based on these functions. For practical purposes, the code below generates the smarter index on a subset of the data, as generating an index with stemming on the entire set would take too much time. (You don't need to change or add anything in the code block below. Just leave it as it is.)

In [32]:
# Below, we create our smarter index (based on a subset of the documents for demonstration purposes)
smarter_index = defaultdict(set)

# Here we define the subset (somewhat arbitrary):
subset_of_ids = set(key for key in Abstracts.keys() if 24000000 <= key < 25000000)
subset_of_abstracts = ((key, Abstracts[key]) for key in subset_of_ids)

# Building our smarter index:
for (id, abstract) in subset_of_abstracts:
    for term in smarter_preprocess(smarter_tokenize(abstract)):
        smarter_index[term].add(id)

Now implement the `smarter_and_query` function, based on the two functions `smarter_tokenize` and `smarter_preprocess` you defined above and accessing our new index `smarter_index`. You can start from the code for `and_query` from the last assignment.

In [37]:
# Smarter and_query based on the smarter tokenize and preprocess functions

# Your code here:

def smarter_and_query(query_string):
    search_words = []
    search_words = smarter_preprocess(smarter_tokenize(query_string))
    display_results = set(smarter_index[search_words[0]])
    for i in search_words:
        temp_results = set()
        for j in smarter_index[i]:
            temp_results.add(j)
        display_results = display_results.intersection(temp_results)
    return list(display_results)

### Task 2

Run the queries "eosinophil signal inflammation" and "eosinophil rhythm" with the new `smarter_and_query` function from task 1. Do they return paper *24037376* (this is our exemplary paper from the last assignment)? For each of the two example queries, what do our new smarter functions specifically contribute to the result (as compared to our previous naive implementations for tokenization and preprocessing)?

In [45]:
# for i in smarter_and_query("eosinophil signal inflammation"):
#     display_summary(i)
    
print(smarter_and_query("eosinophil signal inflammation"))

print(smarter_and_query("eosinophil rhythm"))

[24037376, 24600450, 24645804, 24486607]
[24037376]


**Answer:** [For "eosinophil signal inflammation", in the abstract, the token 'signals' has been turned into the stem 'signal', and hence matches. Further, rather than it being tokenised into 'inflammation,' the smarter tokenisation has turned it into 'inflammation', ','. Which again, allows for a match.

For 'eosinophil rhythm', in the abstract, the token 'rhythms' has been turned into the stem 'rhythm', and hence matches.]

### Task 3

Now we move to a different subject and use our old index again. That is, we **don't** use the smarter functions defined above for tasks 3 to 5!

Create a function `tfidf(t,d)` that returns the tf-idf score of term `t` in document `d` by using `tf(t,d)`, `df(t)` and `num_documents` as defined above. To do this, first implement a function `idf(t)` to calculate the inverse document frequency, and then use this function to calculate the full tf-idf. Use the _add-one-smoothing_ version of idf, so we don't run into problems with terms that don't appear in the collection at all. The relevant formulas can be found on the lecture slides. Use tf-idf with plain (non-logarithmic) term frequency, as applied by scoring variant `ntn`. Test your function with the examples shown below. You can use the `log10(n)` function to calculate the base 10 logarithm.

Again, use our old (non-smart) index for this task and the tasks below, and **not** the functions defined in tasks 1 and 2.

In [53]:
# Your code here:
# tf(t,d): How often does a term occur in a document
# df(t): In how many documents does a term occur
# num_documents: The number of documents in our index
def idf(t):
    return (1+log10(num_documents/df(t)))

def tfidf(t,d):
    return tf(t,d)*idf(t)


print(tfidf('ilc2', 24037376))
print(tfidf('regulation', 24037376))
print(tfidf('with', 24037376))

17.244164888192586
5.986884249901944
2.7198393254411575


### Task 4

Create a function `query_ntn(query_string)`, which accepts as input a single query string of one or more words, and returns or prints a list of (up to) 10 best matching documents, along with their score. Use _tf-idf_ to calculate document scores based on the query, applying variant `ntn`, as above (see the formula for the `ntn` version of scoring on the lecture slides). Use an auxiliary function `score_ntn` to calculate the score. The results should be shown in descending order by score.

You can start by copying your `or_query` function from assignment 2, then expand that to rank the results, making use of the `tfidf(t,d)` function you created above. Make sure to apply term weighting to the query terms too.

Demonstrate your function by giving it the exemplary query string "factors of food allergy for children".

In [70]:
# Your code here:
# higher tf-idf means rarer
# find whether words are in a document, and then rank these terms by their tf-idf (do I sum all words?)
def score_ntn(query_words, doc_id):
    score = 0
    for word in query_words:
        score += tfidf(word, doc_id)

def query_ntn(query_string):
    query_words = []
    query_words = preprocess(tokenize(query_string))
#     print(query_words)
    doc_matches = set(inverted_index[query_words[0]])
    for i in query_words:
        temp_matches = set()
        for j in inverted_index[i]:
            temp_matches.add(j)
        doc_matches = doc_matches.union(temp_matches)
    for doc in doc_matches:
        score_ntn(query_words, doc)
    #store score against id
    #sort based on score
    #return top 10 results
#     return list(doc_matches)

query_ntn("factors of food allergy for children")
# Example query:
#query_ntn("factors of food allergy for children")

2.6618671950292367
20.481032316831307
2.398474969765404
39.9481697021549
5.508351729318432
3.861104679911939
5.0366135971835515
9.85729210432545
6.055411852189255
5.0366135971835515
1.199237484882702
6.762635532593919
1.4626297101465346
7.435088566948956
10.360347987242026
1.199237484882702
24.58180065439599
34.21770908517075
2.398474969765404
12.495430731743596
21.129756783575253
1.199237484882702
7.722209359823878
22.23223324808282
2.6381386274181473
6.73890696498283
9.85729210432545
5.552355969272679
2.8778022850708904
9.377045592400975
1.199237484882702
4.796949939530808
12.796207388583147
2.398474969765404
23.7678954109934
22.20513125511402
6.235851082066254
2.398474969765404
12.495430731743596
3.861104679911939
3.861104679911939
12.232038506479764
8.350658154224668
6.888028444078133
4.100768337564682
5.815748194536512
21.997869235628784
12.338720069477688
27.844935180946994
3.861104679911939
17.747979418968804
5.9961874244135105
7.014985679419214
5.552355969272679
3.2903159894300

12.758822957007432
16.336984953427404
1.4389011425354452
13.548999632798928
5.0366135971835515
7.435088566948956
21.129756783575253
2.6618671950292367
12.255767074090853
23.18037128091995
4.796949939530808
15.466224290131194
5.952183184459263
4.353118484389977
3.529979647082769
13.431275991362465
13.934331874279042
2.6381386274181473
9.0135189214043
7.458817134560045
27.365607865641504
14.701505196574479
7.435088566948956
16.619927636919368
5.0366135971835515
19.667127073428723
4.124496905175771
13.431275991362465
4.936479151450708
4.796949939530808
16.989608326193753
1.4626297101465346
9.86147021370841
7.458817134560045
2.398474969765404
13.934331874279042
6.235851082066254
4.3641605628285145
4.3656904609687945
5.9961874244135105
13.455004558973556
12.164305698914427
2.6381386274181473
7.391084326994708
27.013287963850196
15.590087303475126
8.873989709484402
17.268652103663314
3.7135130989569163
7.722209359823878
13.570805203282365
13.934331874279042
10.57628307728368
14.7853773885803

19.667127073428723
10.096955761978194
14.210410588715426
16.485022545136896
4.616510709653809
19.359730608210636
4.796949939530808
2.398474969765404
5.244959504054599
3.861104679911939
15.373233016814488
10.61269813406732
3.5537082146938586
8.154079539907185
2.9252594202930693
2.398474969765404
9.629395739226272
10.120684329589281
2.398474969765404
12.55465515951934
19.421324043868506
7.195424909296213
4.0770397699535925
18.251954498504364
16.541598761669963
11.115754016983896
10.648193994231976
19.30268120581306
8.750126696140468
5.9961874244135105
3.861104679911939
3.8373761123008494
9.389732081573529
38.201872094401374
31.107142154186207
4.796949939530808
17.57858226753034
12.255767074090853
28.91204494425388
7.195424909296213
5.68879095919543
8.829985469530154
1.199237484882702
7.51804156233579
2.398474969765404
11.032801021597063
1.199237484882702
7.195424909296213
1.199237484882702
1.4626297101465346
2.398474969765404
16.069414618780613
1.199237484882702
7.435088566948956
4.10076

4.912750583839618
12.71136582178525
1.199237484882702
1.199237484882702
20.275300195486352
11.559585472124727
24.726550041604376
2.398474969765404
12.255767074090853
2.6381386274181473
13.694668216626297
11.228460439295558
14.689737904020914
13.455004558973556
6.444196988937302
23.359559816017224
13.958060441890133
20.75783624538272
5.300005822447384
8.634326051831657
6.235851082066254
4.353118484389977
5.9961874244135105
6.1357166363334095
5.552355969272679
7.127692101730876
19.566992627695875
1.199237484882702
9.593899879061617
17.988562273240532
4.796949939530808
7.151420669341965
10.671922561843065
5.9961874244135105
20.387037243005935
11.775520562166381
3.5977124546481063
1.199237484882702
12.396380173396565
7.435088566948956
2.398474969765404
17.304147963827972
6.235851082066254
2.398474969765404
17.703975179014556
8.59032181187741
23.91240741632151
9.209292851696532
21.129756783575253
7.722209359823878
3.861104679911939
3.290315989430026
3.5977124546481063
9.916516532101195
5.03

14.654242043856259
5.276277254836295
8.897718277095489
30.3829393626323
9.209292851696532
29.751919221152626
9.833563536714362
11.535856904513638
3.5977124546481063
33.52105429734011
9.85729210432545
2.398474969765404
5.0603421647946405
18.418585703393063
7.199603018679172
12.015184219819124
5.952183184459263
12.291262934255508
9.593899879061617
15.945551605436679
36.640191825907806
25.038318598709374
7.643434473820003
26.57820856005666
10.432258904190322
2.6381386274181473
9.137381934748234
18.467889588546015
4.968880789618215
3.861104679911939
8.671710483407375
15.157297926772834
6.991257111808125
3.8373761123008494
18.56188466237139
3.8373761123008494
24.007559068646145
9.150068423920786
10.57628307728368
16.931502211520407
8.92144684470658
3.5977124546481063
21.919933459366753
5.0366135971835515
18.01045244761365
28.178593912425107
6.499243307330086
4.616510709653809
7.458817134560045
10.793137363944318
1.4626297101465346
3.861104679911939
2.6618671950292367
13.98251422361625
5.996

...

17.05271701362166
11.992374848827021
1.199237484882702
12.495430731743596
13.910603306667952
9.593899879061617
32.10566473684876
10.096955761978194
8.92144684470658
6.888028444078133
9.593899879061617
12.519159299354687
17.05271701362166
23.0763379291647
14.382787449557302
17.32787653143906
3.8373761123008494
12.71136582178525
17.05271701362166
8.203066573269645
3.861104679911939
2.6381386274181473
11.056529589208152
8.394662394178916
1.199237484882702
17.81916512180207
2.398474969765404
17.267732907044326
10.16886667892649
11.559585472124727
9.593899879061617
16.356535411655535
6.235851082066254
2.6618671950292367
4.796949939530808
8.225265242740452
3.5977124546481063
10.88713243776969
17.13663938380057
13.191612333709724
3.4172732247711077
8.087265928960834
7.698480792212789
6.2595796496773435
6.522971874941176
14.91763426912009
6.71517839737174
2.90153085268198
4.4895534743127286
5.0366135971835515
20.356688867016317
40.814881415032126
9.113653367137143
12.495430731743596
22.5923864

9.137381934748234
13.947018363451594
1.199237484882702
4.673086926186875
2.398474969765404
1.4626297101465346
22.137512960142423
14.566233563947762
1.199237484882702
8.658054619442748
27.62900009090534
3.5977124546481063
11.272464679249806
7.195424909296213
6.522971874941176
3.5977124546481063
1.199237484882702
19.187799758123234
5.0603421647946405
8.394662394178916
17.08821287378632
12.735094389396341
17.988562273240532
5.0603421647946405
9.789559296760112
28.82823757578804
24.287815892465485
15.133569359161744
2.398474969765404
11.388265323558615
2.398474969765404
11.79924912977747
30.9732408629124
8.873989709484402
7.698480792212789
12.519159299354687
2.398474969765404
21.555926351899018
9.833563536714362
15.852560332119975
22.12671504238093
7.458817134560045
32.90619654236062
2.398474969765404
12.255767074090853
11.535856904513638
14.390849818592425
9.24249917388929
3.5977124546481063
33.14494100339438
31.011185817836324
12.123510193551063
7.311225553605023
6.764165430734199
14.770

35.10400678807896
30.722737466017882
10.44173665877199
21.474537680369053
13.693749020007312
11.056529589208152
12.255767074090853
6.522971874941176
16.309078276433358
7.550889211257766
2.398474969765404
31.537367923535502
16.4088958703757
21.988276968453384
6.235851082066254
2.274611956421471
4.0770397699535925
4.796949939530808
1.199237484882702
1.199237484882702
3.861104679911939
7.195424909296213
1.199237484882702
10.268886612065598
8.92144684470658
5.300005822447384
7.014985679419214
18.970945471462596
23.288568095687914
4.796949939530808
21.129756783575253
1.4389011425354452
6.2595796496773435
11.411993891169704
7.026027757857751
12.471702164132507
7.961873017476621
7.458817134560045
14.870177133897911
14.398122149972535
15.205480276110041
7.435088566948956
20.758167743543453
25.50660383557632
2.398474969765404
6.235851082066254
3.5977124546481063
3.5977124546481063
7.458817134560045
7.391084326994708
12.108175493135832
6.499243307330086
4.59278214204272
2.398474969765404
10.0969

1.954643514624573
24.2350591120446
12.499608841126557
17.887508630888703
36.66983074023889
8.658054619442748
8.177808107518274
5.300005822447384
5.9961874244135105
1.199237484882702
2.398474969765404
10.096955761978194
3.8373761123008494
32.335407851878706
9.85729210432545
12.232038506479764
19.234337696726424
11.296193246860895
14.870177133897911
7.698480792212789
2.6618671950292367
11.092025449372807
9.916516532101195
3.5977124546481063
1.4389011425354452
2.6618671950292367
24.61894092529466
1.4389011425354452
14.63051347624517
15.85347952873896
15.590087303475126
1.199237484882702
24.21590497551719
28.54111678291312
22.753275265370657
11.992374848827021
39.83639083315503
30.429916061383423
2.6618671950292367
11.032801021597063
12.255767074090853
16.57247050169719
14.654242043856259
11.992374848827021
14.893905701509002
17.075526384613767
13.455004558973556
9.470036865717685
11.032801021597063
7.435088566948956
8.897718277095489
2.6618671950292367
17.531125132308162
3.837376112300849

23.79162397860449
31.166366581961952
30.698081345095
29.006235106669468
12.040442685570495
5.0366135971835515
19.689936444420823
4.100768337564682
2.398474969765404
12.232038506479764
2.6381386274181473
4.124496905175771
18.28745035866902
6.235851082066254
11.032801021597063
5.0603421647946405
1.199237484882702
10.073227194367103
2.90153085268198
1.199237484882702
3.5977124546481063
49.16557461194029
14.893905701509002
13.730164076790953
7.698480792212789
9.593899879061617
13.067749320365788
12.371567718399664
19.761122147254092
3.5977124546481063
19.210609129115337
9.916516532101195
16.979684560857617
7.195424909296213
9.209292851696532
13.25083676148547
11.032801021597063
4.616510709653809
8.386600025143792
2.398474969765404
2.398474969765404
6.522971874941176
32.00400039297563
1.199237484882702
15.133569359161744
18.491618156157106
5.9961874244135105
10.61269813406732
48.250343918589024
5.0366135971835515
14.173995531931787
13.694668216626297
12.471702164132507
5.0603421647946405
8.

### Task 5

In this last task, you should create a second version of the query function from Task 4, called `query_ntc`. This second version should use, as its name suggests, variant `ntc` instead of `ntn`, and therefore apply the cosine similarity measure, in addition to applying _tf-idf_. For this, consult the formula for variant `nnc` on the lecture slides and adopt it to include the _idf_ metric (that is, add the `t` element of `ntc`). (You can drop the square root of |q| in the formula, as indicated on the slides.) Again, make sure to apply term weighting to the query terms too.

As a first step, we can calculate beforehand the length of all document vectors (because they don't depend on the query) for document vectors consisting of _tf-idf_ values. The code below does just that, assuming that you defined the function `tfidf(t,d)` above (don't change this code block, just run it):

In [45]:
tfidf_length_values = defaultdict(int)

for (doc_id, abstract) in Abstracts.items():
    l = 0
    for t in tf_matrix[doc_id].keys():
        l += tfidf(t,doc_id) ** 2
    tfidf_length_values[doc_id] = sqrt(l)

def tfidf_length(d):
    return tfidf_length_values[d]

We can now get the length of a document vector by calling `tfidf_length(d)`.

Based on this, you can now implement `query_ntc` in the code block below. You should again first define an auxiliary function, called `score_ntc`. You can start by copy-pasting the code from Task 4.

To output the results, use the provided `display_summary` function to make the output a bit more like the results page of a search engine. Lastly, demonstrate your `query_ntc` function with the same example query as above: "factors of food allergy for children"

In [ ]:
# Your code here:

#def score_ntc(query_words, doc_id):
#    ...

#def query_ntc(query_string):
#    ...

# Example query:
#query_ntc("factors of food allergy for children")

# Submission

Submit the answers to the assignment via Canvas as a modified version of this Notebook file (file with `.ipynb` extension) that includes your code and your answers.

Before submitting, restart the kernel and re-run the complete code (**Kernel > Restart & Run All**), and then check whether your assignment code still works as expected.

Don't forget to add your name, and remember that the assignments have to be done individually and group submissions are **not allowed**.